In [68]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nltk
from collections import Counter, deque
import random

In [4]:
def read_data(filename):
    with open(filename, 'r') as f:
        data=[]
        files=f.read()
        files=files.lower()
        files=nltk.word_tokenize(files)
        data.extend(files)
        
    return data

words=read_data('wikipedia2text-extracted.txt')


In [28]:
vocabulary_size = 50000 
count=[('UNK', -1)]
count.extend(Counter(words).most_common(vocabulary_size-1))
vocab2int={i:v for v, i in enumerate([word for word, _ in count])}
int2vocab={i:v for i, v in zip(vocab2int.values(), vocab2int.keys())}
data_ints=[vocab2int.get(val, -1) for val in words]

In [60]:
data_index=0
def generate_batch(batch_size, window_size):
    global data_index
    batch=np.ndarray(shape=(batch_size), dtype=np.int32)
    labels=np.ndarray(shape=(batch_size,1), dtype=np.int32)
    span=2*window_size+1
    num_samples=2*window_size
    buffer=deque(maxlen=span)
    
    for _ in range(span):
        buffer.append(data_ints[data_index])
        data_index=(data_index+1)%len(data_ints)
        
    for i in range(batch_size//num_samples): # m+1<= i <=N-m refer text
        k=0
        for j in list(range(window_size)) + list(range(window_size+1, span)):
            batch[i*num_samples+k]=buffer[window_size]
            labels[i*num_samples+k]=buffer[j]
            k+=1
            
        buffer.append(data_ints[data_index])
        data_index=(data_index+1)%len(data_ints)
        
    return batch, labels

In [92]:
embedding_size=128
batch_size=128
window_size=4
valid_size=16
valid_window=50
num_sampled=32
epochs=100000

valid_examples=np.array(random.sample(range(valid_window), valid_size)) #most commonly occuring words or opp case
valid_examples=np.append(valid_samples, random.sample(range(1000, 1000+valid_window), valid_size), axis=0)#rare words or opp
                                                                                                                #case

In [91]:
train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                        stddev=0.5 / np.sqrt(embedding_size))
)
softmax_biases = tf.Variable(tf.random_uniform([vocabulary_size],0.0,0.01))

embed = tf.nn.embedding_lookup(embeddings, train_dataset)#out shape = shape(parm2)+shape(par1)[:, 1:]
loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(
        weights=softmax_weights, biases=softmax_biases, inputs=embed,
        labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size)
)

optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

In [90]:
norm=tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings=embeddings/norm
valid_embeddings = tf.nn.embedding_lookup(
normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [95]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #for epoch in range(epochs):
    source, targets=generate_batch(batch_size, window_size)
    sim=sess.run(similarity, feed_dict={train_dataset:source, train_labels:targets})
    print(sim.shape)


(48, 50000)


In [89]:
skip_losses=[]
avg_loss=0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        source, targets=generate_batch(batch_size, window_size)
        _, loss=sess.run([optm, loss], feed_dict=train_dataset:source, train_labels:targets)
        avg_loss+=loss
        
        if (epoch+1)%2000==0:
            avg_loss/=2000
            print("Avg. loss at step %d = %f", %(epoch+1, avg_loss))
            skip_losses.append(avg_loss)
            avg_loss=0
            
        if (epoch+1)%10000==0:
            sim=similarity.eval()
            for i in range(valid_size):
                valid_word=int2vocab[valid_dataset[i]]
                top_k=8
                nearest=
                for k in range(top_k):
                    
            
    

[[5.2474678e-01 1.1076356e-01 3.9347810e-01 ... 1.1921988e-01
  2.0631781e-01 1.1011354e-01]
 [6.1760038e-01 6.0697520e-01 1.2021061e-02 ... 8.5996085e-01
  4.3269670e-01 1.4159871e-05]
 [8.7795031e-01 1.3306703e-01 3.8104516e-02 ... 6.6567194e-01
  4.1667182e-02 3.1872335e-01]
 ...
 [3.4354240e-01 7.6964991e-03 6.8699151e-02 ... 3.6620507e-01
  2.7619868e-05 4.8523460e-02]
 [7.5176734e-01 1.3101089e-04 2.8701469e-01 ... 4.0340790e-01
  9.2573875e-01 8.6790049e-01]
 [2.1000512e-01 2.7518719e-01 2.3639007e-01 ... 9.2676628e-01
  5.9659201e-01 4.1742063e-01]]
